Para realizar la optimización de un portafolio en Python, necesitamos datos históricos de precios de las acciones. Podemos obtener estos datos de varias fuentes, como Yahoo Finance, utilizando la biblioteca `pandas_datareader`. Luego, podemos usar la biblioteca `scipy` para realizar la optimización.

Aquí hay un ejemplo simple de cómo podríamos hacer esto:


In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
from scipy.optimize import minimize

# Obtener datos de precios
def get_data(tickers, start_date, end_date):
    stock_data = pd.DataFrame()
    for t in tickers:
        stock_data[t] = web.DataReader(t, 'yahoo', start_date, end_date)['Adj Close']
    return stock_data

# Calcular rendimientos diarios
def compute_daily_returns(stock_data):
    return np.log(stock_data / stock_data.shift(1))

# Función objetivo para minimizar
def objective(weights, returns, risk_tolerance):
    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    return -portfolio_return / (risk_tolerance * portfolio_volatility)

# Restricciones y límites
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for x in range(3))

# Tickers, fechas y tolerancia al riesgo
tickers = ['TSLA', 'MSFT', 'NVDA']
start_date = '2020-01-01'
end_date = '2022-12-31'
risk_tolerance = 0.05  # Alta tolerancia al riesgo

# Obtener datos y calcular rendimientos
stock_data = get_data(tickers, start_date, end_date)
returns = compute_daily_returns(stock_data)

# Optimización
initial_guess = [1/3, 1/3, 1/3]  # Comenzar con una distribución uniforme
optimal_portfolio = minimize(objective, initial_guess, args=(returns, risk_tolerance), method='SLSQP', bounds=bounds, constraints=constraints)

print("Pesos óptimos del portafolio:", optimal_portfolio.x)


Este código obtiene los precios de cierre ajustados de las acciones de Tesla, Microsoft y Nvidia desde el 1 de enero de 2020 hasta el 31 de diciembre de 2022. Luego calcula los rendimientos diarios de estas acciones y utiliza la optimización para encontrar la combinación de pesos que maximiza el rendimiento del portafolio para una alta tolerancia al riesgo.

Por favor, ten en cuenta que este es un ejemplo simplificado. En la práctica, la optimización de portafolios puede ser mucho más compleja e involucrar muchas más consideraciones, como las comisiones de transacción, las restricciones de inversión, las proyecciones de rendimiento futuro, entre otros. Además, este código puede no funcionar tal cual, ya que depende de la disponibilidad de los datos de Yahoo Finance y de las bibliotecas de Python mencionadas.